In [8]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [9]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [10]:
from test_db_schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
)

In [11]:
from typing import Any


async def dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [12]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    match_by=User.name,
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [13]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [14]:
with cProfile.Profile() as pr:
  db = DataBase()
  rec = await data_source.load([data], db=db)

  stats = pstats.Stats(pr)

Async-loading `Search`:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
db.engine

Engine(sqlite:///file:c5f637e5a2?cache=shared&mode=memory&uri=true)

In [16]:
list(db[User.name].items())


[
    ('5d4f5fe9-5e9b-4db4-9592-f02e53fdfc91', 'John'),
    ('6f26fd54-db4d-4e75-b7cd-9b1dfc9a2190', 'John'),
    ('899f9810-1aba-4575-8ea5-e2d64fd7e5e8', 'John'),
    ('93fd36bb-1b96-41e3-b8f2-160ec38f508c', 'Jane'),
    ('948821d7-dbed-4720-97fd-4832b51edb68', 'John'),
    ('b8005cee-c106-4977-bd26-2a42f3f3cfa7', 'Jane'),
    ('b8491192-aaf9-4a87-a440-bf1c0e567cb8', 'Jack')
]

In [17]:
print(db[Search].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_Search".result_count AS result_count
FROM "test_db_schema_Search"


In [18]:
print(db[Search.term].select_str)

SELECT "test_db_schema_Search".term AS term
FROM "test_db_schema_Search"


In [19]:
print(db[Search.result_count].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_Search".result_count AS result_count
FROM "test_db_schema_Search"


In [20]:
print(db[Search.results].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_SearchResult"."test_db_schema_Search_term" AS "test_db_schema_Search_term",
       "test_db_schema_SearchResult"."test_db_schema_Project_number" AS "test_db_schema_Project_number",
       "test_db_schema_Project".name AS name,
       "test_db_schema_Project".start AS
START, "test_db_schema_Project"."test_db_schema_Organization__id" AS "test_db_schema_Organization__id",
       "test_db_schema_Project".number AS number,
       "test_db_schema_Project"."end" AS "end",
       "test_db_schema_Project".status AS status
FROM "test_db_schema_Search"
JOIN "test_db_schema_SearchResult" ON "test_db_schema_SearchResult"."test_db_schema_Search_term" = "test_db_schema_Search".term
JOIN "test_db_schema_Project" ON "test_db_schema_SearchResult"."test_db_schema_Project_number" = "test_db_schema_Project".number


In [21]:
db[Search.term].values()

['test']

In [22]:
list(db[Search.result_count].items())

[('test', 3)]

In [23]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         769018 function calls (721094 primitive calls) in 1.652 seconds

   Ordered by: cumulative time
   List reduced from 1757 to 143 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    50/47    0.002    0.000    0.843    0.018 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:570(_load_record)
       37    0.000    0.000    0.799    0.022 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:733(_load_rec_from_item)
       33    0.001    0.000    0.620    0.019 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:4330(__init__)
       34    0.000    0.000    0.613    0.018 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1367(__ior__)
       34    0.001    0.000    0.611    0.018 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2790(_mutate)
       34    0.001    0.000    0.571    0.017 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.p